Note to self 6/4-22:
Totalt antall stemmer godt med PSAK120.
Alt med DP stemmer bra med excel, men ikke så bra med PSAK120.
Automatisk stemmer så bra at autograd blir det samme på figur 1, men litt unøyaktighet her også.


For the future 22/4-22:
Når vi joiner med `PEN.T_SKJEMA` forsvinner (nesten) alle ikke-selvbetjente saker. Teste ut andre typer join for å konservere antall saker. Hvordan håndtere resulterende nullverdier fra `PEN.T_SKJEMA`?
Kan være nødvendig å se mer på `OPPRETTET_AV`; mye der som kommer fra batch.

In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils

importlib.reload(pandas_utils)

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()
resource_name = f"projects/knada-gcp/secrets/vebjorns-hemmelighet/versions/latest"
secret = secrets.access_secret_version(name=resource_name)
secrets = dict([line.split("=") for line in secret.payload.data.decode('UTF-8').splitlines()])

ORACLE_HOST = '10.53.136.15'
ORACLE_PORT = '1521'
ORACLE_SERVICE = 'pen'

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

In [ ]:
df_v1 = pandas_utils.pandas_from_sql('../sql/v1.sql', con)

In [ ]:
con.close()

In [ ]:
def get_dp_behandling(df, behandling):
    return df[df["K_BEHANDLING_T"] == behandling]

In [ ]:
df_v1.tail(16)

In [ ]:
df = df_v1.copy()
# Add "ÅR-MÅNED" TO FRONT
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

In [ ]:
df = df.merge(df.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
                left_on="ÅR-MÅNED",
                right_on="ÅR-MÅNED",
                suffixes=(""," TOTALT")
        )


In [ ]:
df["ANDEL"] = df.ANTALL.divide(df["ANTALL TOTALT"])

# Prep for plott

In [ ]:
nav_colors = (
    "#0067C5",
    "#BA3A26",
    "#06893A",
    "#634689",
    "#FF9100",
    "#66CBEC",
    "#F0C419",
)
pastel = px.colors.qualitative.Pastel

figs = []

### Automatiseringsgrad

In [ ]:
df_auto = df.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_auto = df_auto[df_auto.AUTOMATISERING == "AUTO"].reset_index(drop=True)
df_auto["ANDEL_PROSENT"] = df_auto["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_auto.tail()

In [ ]:
n_months = 17

df_plot = df_auto.iloc[-n_months:-1,:]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[0], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisk", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[0.2,0.8], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs.append(fig)

### Selvbetjening

In [ ]:
df_selv = df.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "OPPRETTET_AV", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_selv = df_selv[df_selv.OPPRETTET_AV == "bruker"].reset_index(drop=True)
df_selv["ANDEL_PROSENT"] = df_selv["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_selv.tail()

In [ ]:
df_plot = df_selv.iloc[-n_months:-1,:]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[1], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Selvbetjent", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel selvbetjent", secondary_y=True, range=[0.5,1], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs.append(fig)

In [ ]:
for fig in figs:
    fig.show()

# Publiser datastory

In [ ]:
from datastory import DataStory

In [ ]:
ds = DataStory(f"Automatiserings- og selvbetjeningsgrad for alderspensjon")

ds.markdown("**(Under utvikling)**")
ds.header(f"Automatisering av førstegangsbehandlingssaker for alderspensjon i pesys. Saker opprettet av batch er eksludert.", level=2)
ds.plotly(figs[0].to_json())

ds.header(f"Selvbetjening av førstegangsbehandlingssaker for alderspensjon i pesys. Saker opprettet av batch er eksludert.", level=2)
ds.plotly(figs[1].to_json())

In [ ]:
ds.publish(url="https://nada.intern.nav.no/api") #prod